In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')


In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Unsupervised Machine Learning  and Clustering


<h2 id="tocheading">Table of Contents</h2>
<div id="toc"></div>

$$
% vectors and matrices
\def\v#1{\boldsymbol{#1}}
\def\m#1{\boldsymbol{#1}}
\def\p#1{\mathtt{#1}}
% confusion matrix stuff
\def\TP\mathit{TP}
\def\TN\mathit{TN}
\def\FP\mathit{FP}
\def\FN\mathit{FN}
% statistics 
\DeclareMathOperator\mse{mse}
\DeclareMathOperator\E{E}
\DeclareMathOperator\Var{Var}
\DeclareMathOperator\Cov{Cov}
\DeclareMathOperator\Bias{Bias}
\DeclareMathOperator*{\argmax}{arg\,max}
\DeclareMathOperator*{\argmin}{arg\,min}
$$



The story so far:

- Linear Discriminant Analysis (LDA) and Fisher's linear discriminant
- Principal Component Analysis (PCA)
- Feature Selection
- Supervised Learning


Notations:

* Uppercase letters such as $X$ or $Y$ denote generic aspects of a variable (i.e. the actual random variable)
* Observed values are written in lowercase. The ith observed value of $X$ is written as $x_i$
* Matrices are written in bold uppercase letters as in $\m{X}$
* Observations map as *rows* in the matrix while the observed variables are the *columns*.

So if I measure two observables $p = 2$ the size and weight of $N = 100$ people, I get a $N \times p$ matrix $\v{X}$.
One observation in that matrix is denoted as $x_i = [ \mathrm{size}, \mathrm{weight} ]$ while all observations of the variable weight are denoted by $\v{x}_{\bullet 1}$, analogous to the numpy indexing `X[:, 1]`

So far we have been occupied with 
predicting the values of one or more outputs or response variables $Y = (Y_1, \ldots, Y_m)$ for a given set of input or predictor variables $X = (X_1, \ldots , X_p)$. 

One possible definition of supervised machine learning

> Given a $N \times p$ matrix $\v{X}$ and some associated output vector $\v{Y} \in \mathbb{R}^N$,
 find a function $f(X) = \hat{Y}$ that takes a vector $X \in \mathbb{R}^p$ and returns a prediction for $\hat{Y}$
 where some "loss function" $L(Y, f(X))$ is minimized for all $X$.
 
We've tried to split points into different classes by finding a decision function which seperates the points in an "optimal" way. This process is often called __classification__

In __regression__ the dependent variable is not discrete but a continous value. The problem remains similar however. From *known* input data we try to find a function which accurately predicts $Y \in \mathbb{R}$.

In __unsupervised__ machine learning problems the output vector $\v{y}$ is *unknown*.

In [ ]:
from ml import plots
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.colors import ListedColormap

In [ ]:
pd.options.display.max_rows = 10
plots.set_plot_style()


def create_discrete_cmap(k, under='k'):
    cmap = ListedColormap([f'C{i}' for i in range(k)])
    cmap.set_under('k')
    return cmap

In [ ]:
%matplotlib widget

## Classification Example (Iris)

The sterotypical machine learning data set. Given some measurements from flower petals is it possible to deduce the species for a new sample of measurements?

Originally published in 1936 by Ronald Fisher in his paper about LDA. 

<table style="width:100%">
  <tr>
    <td><img src="./ml/images/setosa.jpg" alt="Drawing" style="width: 200px;"/></td>
    <td><img src="./ml/images/versicolor.jpg" alt="Drawing" style="width: 200px;"/></td> 
    <td><img src="./ml/images/virginica.jpg" alt="Drawing" style="width: 200px;"/></td>
  </tr>
</table>

<p style="color:gray"> Iris flower data set. (2017, October 24). In Wikipedia, The Free Encyclopedia.  <p>

Length and width of the petals and sepals making the dataset four dimensional.  Measurements where taken in Canada by Botanist Edgar Anderson:

> all from the same pasture, and picked on the same day and measured at the same time by the same person with the same apparatus

The label vector $\v{y}$ is known because the botanist categorized the flowers into species.

In [ ]:
from sklearn.datasets import load_iris
from itertools import combinations


iris = load_iris()
X = iris.data  # we only take the first two features.
y = iris.target

cmap = create_discrete_cmap(len(iris.target_names))

f, axs = plt.subplots(3, 2, constrained_layout=True)
axs = axs.flatten()

for i, (col_x, col_y) in enumerate(combinations(range(X.shape[1]), 2)):

    plot = axs[i].scatter(
        X[:, col_x], X[:, col_y],
        c=y,
        cmap=cmap,
        label='species',
        vmin=-0.5, vmax=len(iris.target_names) - 0.5,
    )
    axs[i].set_xlabel(iris.feature_names[col_x].replace('(cm)', '/ cm'))
    axs[i].set_ylabel(iris.feature_names[col_y].replace('(cm)', '/ cm'))
    
bar = f.colorbar(plot)
bar.set_ticks(np.arange(len(iris.target_names)))
bar.set_ticklabels(iris.target_names)

The question this data set can help us answer is:

> What species does this flower likely belong to.

## Unsupervised Learning

In unsupervised learning we have no given $Y$.

These methods try to find the underlying (joint) probability density $P(X)$ so that we might learn some properties about it.

One common question is whether $X$ is created by a mixture of two or more underlying random variables.

In simpler terms:

 - Given some points, can we infer something about the underlying distributions or labels


### Clustering

Clustering algorithms try to find modes of $P(X)$ based on densities, neighborhood relations or any other measure of 'similarity'  between points.

More generally speaking to quote wikipedia again:

> Cluster analysis or clustering is the task of grouping a set of objects in such a way that objects in the same group (called a cluster) are more similar (in some sense or another) to each other than to those in other groups (clusters).


In the example below an unknown number of gaussian distributions are sampled to create a scatter plot.

Can one infer $P(X)$ by looking at $X$, i.e. the blue dots?

In this case we know that this distribution of blue dots, $X$, was created by sampling $k$ two-dimensional gaussians with known standard deviation.
This is just what 

    X, y = make_blobs(n_samples=300, centers=k, center_box=(-2, 2), cluster_std=0.5)
 
does.
We even know in what region of space we have to look for to find the centroids centroids of these blobs. 

Questions: 

   - How many blobs do we have?
   - Which points belong to which?

In [ ]:
from sklearn.datasets import make_blobs

rng = np.random.default_rng(2)


# choose a random number between 1 and 3
k = rng.integers(1, 4)

# create k blobs 
X, y = make_blobs(n_samples=300, centers=k, center_box=(-2, 2), cluster_std=0.5)

# plot points without any color coding.
fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.scatter(X[:, 0], X[:, 1])
ax.set_axis_off()
None

<br>
<br>
<br>
<br>
<br>
<br>






If you guessed k=2, you're wrong.

In [ ]:
cmap = create_discrete_cmap(k)

fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap)
ax.set_axis_off()
None

### K-Means Algorithm

The k-Means algorithm tries to find a partition of the data into $k$ clusters $S = \{S_1, \ldots, S_k\}$  which minimize the variance within those clusters. The number of clusters $k$ has to specified by the user.

Formally speaking the algorithm solves
$$
{\underset {S}{\operatorname {arg\,min} }}\sum _{S_i \in S}\sum _{x \in S_{i}}\left\|x -{\overline{x}}_{S_i}\right\|^{2}.
$$


This problem is similar in nature to that of feature selection. A minimization over all possible subsets of the points $S$. Solving the general optimization problem is NP-Hard and therefore intractable. 

In simpler terms, it would take too long to find the optimal solution in the general case. 
Again there is a popular greedy heuristic which is usualy used to solve the problem.

#### Loyds Algorithm

Loyds Algorithm (sometimes also simply called *the* k-means algorithm) finds a local optimum using a greedy heuristic.

It does so iterativly according to the following steps 

1. Pick some initial cluster means (or centroids) $\{m_1, \ldots, m_k \}$ either randomly or according to some heuristic.

2. Create a partition $S$  by assigning each point $x \in X$ to the cluster $S_i$ where the distance to $m_i$ is the smallest.

3. Update the cluster means by calculating the means within the assigned clusters. 

4. Repeat steps 2 and 3 until convergence.

One popular convergence is the total change in cluster centroids per iteration. If the cluster centroids do not change from one iteration to the next, return the cluster centroids.


The example below shows a k-mean clustering on $k=3$ random blobs.


In [ ]:
from sklearn.cluster import KMeans

k = 3
cmap = create_discrete_cmap(3)

# create three random blobs
X, y = make_blobs(n_samples=300, centers=k, center_box=(-2, 2), cluster_std=0.5, random_state=1234)

# use KMeans to predict clusters for each sample in X
kmeans = KMeans(n_clusters=3)
prediction = kmeans.fit_predict(X)


# match colors
true_colors = [np.argmax(np.bincount(y[prediction == i])) for i in range(k)]

# shift labels to get the right colors
prediction = np.choose(prediction, true_colors)

fig, ax = plt.subplots()
ax.set_aspect(1)
# plot shades with predicted clusters
ax.scatter(X[:, 0], X[:, 1], c=prediction, cmap=cmap, alpha=0.3, s=250, lw=0, label='prediction')

# plot points with true cluster associations
ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap, label='truth')
ax.legend(loc='upper right')
ax.set_axis_off()

for center in kmeans.cluster_centers_:
    ax.plot(center[0], center[1], 'kX', ms=10)

None

Below you can see an animation of k-Means as it converges. The black hexagons indicate the cluster centroids in the corresponding iteration. 

Create the video by running 

    python ani_kmeans.py

In [ ]:
%%HTML
<video width="1280" height="720" style="max-width: 100%;" controls>
  <source src="kmeans_clustering.mp4" type="video/mp4">
</video>

If the number of clusters in the center is not known, the algorithm does not produce meaningful results.
This is a serious limitation as the number of clusters in the data is rarely known.  


In the example below we sample two uniform 2D distributions and use k-means to cluster them into $k=4$ regions.


In [ ]:
u1 = rng.uniform(-1, 0, size=(500, 2))
u2 = rng.uniform(0, 1, size=(500, 2))
X = np.append(u1, u2, axis=0)

fig, ax = plt.subplots()
ax.scatter(X[:, 0], X[:, 1], color='gray')
ax.set_axis_off()
ax.set_aspect(1)
None

In [ ]:
# use KMeans to predict clusters for each sample in X using the deliberatley wrong value of k=4
k = 4
prediction = KMeans(n_clusters=k).fit_predict(X)
cmap = create_discrete_cmap(k)

fig, ax = plt.subplots()
ax.scatter(X[:, 0], X[:, 1], c=prediction, cmap=cmap)
ax.set_axis_off()
ax.set_aspect(1)
ax.set_title(f'k = {k}')
None

The k-Means algorithm works well on convex clusters with similar standard deviations. But it fails on elongated or concave shapes.

Like all cluster algorithms k-Means has a number of advantages and disadvantages

Pros:

 - Relatively fast
 - Predictable results when inital cluster centroids are fixed.
 - works well on convex and 'blob' like shapes

Cons:

 - Number of clusters has to be known beforehand
 - Gets worse and slower in high dimensions
 
The follwing snippet transforms two random blobs to be elongated along the x axis.
In this case k-Means fails as the neighborhood relation does not yield meaningful results.

In [ ]:
k = 2
X, y = make_blobs(n_samples=1300, centers=k, random_state=2)

transformation = [[20, 0], [0, 1]]
X = np.dot(X, transformation)
prediction = KMeans(n_clusters=2,).fit_predict(X)

fig, ax = plt.subplots()
ax.scatter(X[:, 0], X[:, 1], alpha=0.5, lw=0, c=prediction, cmap=create_discrete_cmap(k))
ax.set_aspect('equal', 'datalim')
None

### Data Scaling

k-Means and many other algorithms, especially the ones using kernel functions, are sensitive to data scaling and centering.

The scikit-learn toolkit includes a  preprocessing module with  methods to scale and transform data.

#### Standard Scaling

The simplest scaler is the `StandardScaler` scaling data to unit variance and zero mean. It makes it 'gaussian like'.

#### MinMax Scaling

Forces the values of each attribute in the data to be within the given feature range. When transforming  to values between 0 and 1 the applied transformation is

$$
\mathbf{x}_j^{\prime} = \frac{(\mathbf{x}_j - \min(\mathbf{x}_j))}{(\max(\mathbf{x}_j) - \min(\mathbf{x}_j))}
$$


#### Quantile Scaling

Transform the given data to follow a normal, uniform or any other distribution with a CDF that can be inverted.
Use inverse transform sampling to transform the data. I.e. apply the inverse CDF to each column in the data.

https://en.wikipedia.org/wiki/Inverse_transform_sampling


In [ ]:
from sklearn import preprocessing

cmap = create_discrete_cmap(k)

fig, axs = plt.subplots(4, 1, figsize=(9, 20), sharex=True)

scalers=[
        preprocessing.StandardScaler(),
        preprocessing.MinMaxScaler(feature_range=(-1, 1)),
        preprocessing.MaxAbsScaler(),
        preprocessing.QuantileTransformer()
]

for scaler, ax in zip(scalers, axs):
    X_prime = scaler.fit_transform(X)
    prediction = KMeans(n_clusters=2, random_state=0).fit_predict(X_prime)
    ax.set_title(scaler.__class__.__name__)
    ax.scatter(X_prime[:, 0], X_prime[:, 1], c=prediction, cmap=cmap, alpha=0.5, lw=0)
    ax.set_aspect(1)
    
None

### Image Compression Using k-Means

For a typical color, or RGB,  image the value for each color component must be stored. It is basically a list of color vectors 

$$
c_i = \begin{pmatrix} R, G, B \end{pmatrix}
$$

For a $420 \times 640$ pixel image that results in  $420 \cdot 640 \cdot 3 \; \text{byte} = 793.8 \;  \text{kilobyte}$ of memory.
This is how a typical Bitmap Image works.

The GIF format allows the compression of color information.  

In the example below we use k-Means to compress the color information of an image. We cluster the space of color components (either RGB, HSV or similar) into $k$ cluster centroids. 
Then only the association of each pixel to the cluster centroids and the centroids themselves have to be stored. 

When clustering the colors of an image into $k=50$ centroids for example  we only need to store $420 \cdot 640 + k \cdot 2 \cdot 3  \; \text{byte} = 269.1 \; \text{kilobyte}$

In [ ]:
from sklearn.datasets import load_sample_image
from skimage import color

#load an example image as a 3D array
image_rgb = load_sample_image("flower.jpg")
image_rgb = np.array(image_rgb, dtype=np.float64) / 255

#store width length and number of colors
width, length, channels = image_rgb.shape  

# show image
plt.figure()
plt.imshow(image_rgb)

#convert image to hsv for nicer plots
image_hsv = color.rgb2hsv(image_rgb)

In [ ]:
# plot H, V and S values of each pixel into a 3D coordinate system.

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection='3d')

ax.scatter3D(
    image_hsv[:, :, 0],
    image_hsv[:, :, 1],
    image_hsv[:, :, 2],
    c=image_rgb.reshape(-1, 3),
    alpha=0.5,
    lw=0,
)

ax.set_xlabel('hue')
ax.set_ylabel('saturation')
ax.set_zlabel('lightness value')
None

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection='3d')

ax.scatter3D(
    image_rgb[:, :, 0],
    image_rgb[:, :, 1],
    image_rgb[:, :, 2],
    c=image_rgb.reshape(-1, 3),
    alpha=0.5,
    lw=0,
)
ax.set_xlabel('red')
ax.set_ylabel('green')
ax.set_zlabel('blue')
None

In [ ]:
from sklearn.utils import resample

# perform k-means on a small random sample of pixels
# we could use all pixels here but it would take much too long.
# we reshape the image into a 2D array for resampling and k-means fitting
flattened_image = image_hsv.reshape(-1, 3)

sample = resample(flattened_image, n_samples=10000, replace=False)
# sample = flattened_image


# get the desired number of cluster centers
kmeans = KMeans(n_clusters=50).fit(sample)
centroids = kmeans.cluster_centers_

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 2))

centroids_rgb = color.hsv2rgb(centroids.reshape(1, -1, 3)).reshape(-1, 3)

ax.scatter(
    np.linspace(0, 1, len(centroids_rgb)),
    np.ones(len(centroids_rgb)),
    c=centroids_rgb,
    s=10000.
)
ax.margins(0.1)
ax.set_axis_off()
None

In [ ]:
# associate each pixel with the number of a cluster (i.e. the nearest centroid/color)
labels = kmeans.predict(flattened_image)

# get the actual value for each cluster centroid and reshape into a 3D image
reconstructed  = centroids[labels].reshape(width, length, channels)

# convert to RGB and plot again.
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

ax1.set_title('Compressed Using KMeans')
ax1.imshow(color.hsv2rgb(reconstructed))

ax2.set_title('Original')
ax2.imshow(image_rgb)

for ax in (ax1, ax2):
    ax.set_axis_off()

### Sorting colors using PCA

When plotting the image centroids there are various ways to arrange the colors. There is no natural order 
to colors. Just as there is no natural order to most things with more than 1 dimension. 

We can try a few different things and try to get a nice output.

1. Order by hue, Value, Saturation in HSV colorspace
2. Order by red, green, blue in RGB colorspace
3. Order by *length* of vector in e.g. eucledian norm 

In [ ]:
flattened_image = image_hsv.reshape(-1, 3)
sample = resample(flattened_image, n_samples=10000, replace=False)

# get the desired number of cluster centers
kmeans = KMeans(n_clusters = 100).fit(sample)

hsv = kmeans.cluster_centers_
rgb = color.hsv2rgb(hsv.reshape(1, -1, 3)).reshape(-1, 3)

In [ ]:
fig, axs = plt.subplots(6, 1, figsize=(12, 12))


for i, label, ax in zip(range(3), 'rgb', axs[:3]):
    
    idx = np.argsort(rgb[:, i])
    
    ax.scatter(np.linspace(0, 1, len(idx)), np.ones_like(idx),  c=rgb[idx], s=10000)
    ax.set_axis_off()
    ax.set_title(f'Order by {label}')
    ax.margins(0.1)
    

for i, label, ax in zip(range(3), 'hsv', axs[3:]):
    
    idx = np.argsort(hsv[:, i])
    
    ax.scatter(np.linspace(0, 1, len(idx)), np.ones_like(idx),  c=rgb[idx], s=10000)
    ax.set_axis_off()
    ax.set_title(f'Order by {label}')
    ax.margins(0.1)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6))


norm = np.linalg.norm(rgb, axis=1)
idx = np.argsort(norm)

ax1.scatter(np.linspace(0, 1, len(idx)), np.ones_like(idx),  c=rgb[idx], s=10000)
ax1.axis('off')
ax1.set_title('Order by norm RGB')
ax1.margins(0.1)

norm = np.linalg.norm(centroids, axis=1)
idx = np.argsort(norm)
ax2.scatter(np.linspace(0, 1, len(idx)), np.ones_like(idx),  c=rgb[idx], s=10000)
ax2.axis('off')
ax2.set_title('Order by norm HSV')
ax2.margins(0.1)
None

We need some other way to reduce the information down to 1D so we can sort it. 
Previously we used the PCA to do just that.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
fig, (ax1, ax2,) = plt.subplots(2, 1, figsize=(12, 6))

c = PCA(n_components=1).fit_transform(rgb).ravel()
idx = np.argsort(c)

ax1.scatter(np.linspace(0, 1, len(idx)), np.ones_like(idx),  c=rgb[idx], s=10000)
ax1.axis('off')
ax1.set_title('Order by PCA transformed RGB.')
ax1.margins(0.1)

c = PCA(n_components=1).fit_transform(hsv).ravel()
idx = np.argsort(c)

ax2.scatter(np.linspace(0, 1, len(idx)), np.ones_like(idx),  c=rgb[idx], s=10000)
ax2.axis('off')
ax2.set_title('Order by PCA transformed HSV')
ax2.margins(0.1)
None

## Density Based Clustering

K-Means only takes distances between neighbours into acount. This works well for convex distributions that are approximately round in shape. Below you'll find another example where k-Means fails miserably. The moons are neither round nor convex yet easily separated into clusters by eye. A simple transformation will not help in this case.

In [ ]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=400, noise=0.1, random_state=1234)

kmeans =  KMeans(n_clusters=2, random_state=0)
prediction =kmeans.fit_predict(X)

cmap = create_discrete_cmap(2)

plt.figure()
plt.axes(aspect=1)
plt.scatter(X[:, 0], X[:, 1], c=prediction, s=200, lw=0, alpha=0.5, cmap=cmap)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap)
plt.axis('off')
plt.plot(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 'kX', ms=10)


None

### DBSCAN

Density-based spatial clustering of applications with noise (DBSCAN) is a clustering method based on the density of different regions within the parameter space. It clusters together points within regions of high density and labels points as outliers that do not lie within a dense region.

1. Start by selecting *core points*. These are all points that have at least $m$ points in their neighbourhood region of radius $\epsilon$.

2. For each core point find connected points to build a cluster $C$. A point is connected to $C$ if its within the $\epsilon$  neighbourhood of any point in $C$ and is also a core point.

3. Assign each point which is not a core point to the nearest cluster with distance being at most $\epsilon$ 



In [ ]:
from sklearn.datasets import make_moons, make_checkerboard, make_circles
from sklearn.cluster import DBSCAN

cmap = ListedColormap([f'C{i}' for i in range(2)])
cmap.set_under('k')

f, [top, center, bottom] = plt.subplots(3, 4, sharex=True, sharey=True)

X_moon, y_moon = make_moons(noise=0.05, random_state=0)
X_circle, y_circle = make_circles(noise=0.05, factor=0.4, random_state=0, n_samples=200)
X_blobs, y_blobs = make_blobs(centers=2, center_box=(-0.5, 0.5), cluster_std=0.4, random_state=0)
X_long, y_long = make_blobs(centers=2, center_box=(-2.1, 2.1), cluster_std=0.1, random_state=0)

data = [(X_moon, y_moon), (X_long, y_long), (X_circle, y_circle), (X_blobs, y_blobs)]

for ax, (X, y) in zip(top, data):
    ax.scatter(X[:, 0], X[:, 1], c=y, s=10, cmap=cmap, vmin=-0.5, vmax=1.5)
    ax.axis('off')
    ax.set_aspect(1)
    
for ax, (X, y) in zip(center, data):
    prediction = KMeans(n_clusters=2, random_state=0).fit_predict(X)
    ax.scatter(X[:, 0], X[:, 1], c=prediction, s=10, cmap=cmap, vmin=-0.5, vmax=1.5)
    ax.axis('off')
    ax.set_aspect(1)
    
for ax, (X, y) in zip(bottom, data):
    prediction = DBSCAN(eps=0.339).fit_predict(X)
    ax.scatter(X[:, 0], X[:, 1], c=prediction, s=10, cmap=cmap, vmin=-0.5, vmax=1.5)
    ax.axis('off')
    ax.set_aspect(1)
    
top[1].set_title('True Clusters')
center[1].set_title('K - Mean Clusters')
bottom[1].set_title('DBSCAN Clusters')

None

### Evaluating Cluster Performance

In the case of unsupervised learning there is no ground truth to which the cluster structure can be compared. 

Some heuristic has to be applied to measure how well a clustering performed. 

#### Silhuette Coefficent 

This evaluation ciriterion assumes a clustering is 'good' if the clusters are dense instead of sparse. 

Define $a$ as the mean distance between a single point $x_0$ and all other points in the cluster $S_p$ with $x_0 \in S_p$.

$$
a(x_0) = \frac{1}{|S_p|} \sum_{x_i \in S_p} \left\|x_i - x_0 \right\|
$$

and $b$ as the mean distance between $x_0$ and all the points in the *nearest cluster* $S_p^\prime$ of which $x_0$ is not a member

$$
b(x_0) = \frac{1}{|S_p^\prime|} \sum_{x_i \in S_p^\prime} \left\|x_i - x_0 \right\|
$$

The Silhuette Coefficent is then defined as 

$$
s = \frac{b - a}{\text{max}(a, b)}
$$


The coefficent takes a value close to +1 for dense clustering and -1 for sparse clusters. 

Unfortunately it doesn't work very reliable.

In [ ]:
from sklearn.metrics import silhouette_score

X, y = make_moons(n_samples=360, noise=0.1, random_state=172)

km = KMeans(n_clusters=2)
prediction_kmeans = km.fit_predict(X)
score_kmeans = silhouette_score(X, km.labels_) 

db = DBSCAN(eps=0.14)
prediction_db = db.fit_predict(X)
score_db = silhouette_score(X, db.labels_) 

fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(10, 5))
ax1.set_title('k-Means clustering score: {:0.3f}'.format(score_kmeans))
ax1.scatter(X[:, 0], X[:, 1], c=prediction_kmeans, cmap=cmap, vmin=-0.5, vmax=1.5)
ax1.axis('off')

ax2.set_title('DBSCAN clustering: {:0.3f}'.format(score_db))
ax2.scatter(X[:, 0], X[:, 1], c=prediction_db, cmap=cmap, vmin=-0.5, vmax=1.5)
ax2.axis('off')
None

## Randomly Sampling Cluster Centers

If we have the correct number of clusters and there are clear clusters, the kMeans algorithm should be stable. 

We can do many kMeans iterations with different initial centers and check if this is the case.

In [ ]:
from tqdm.auto import tqdm

X, y = make_blobs(n_samples=300, centers=3, cluster_std=2, random_state=1234)


centers = {}

ks = range(1, 7)
n_iter = 100
bar = tqdm(total=len(ks) * n_iter)

for k in ks:
    centers[k] = []
    for i in range(100):
        bar.update(1)
        kmeans = KMeans(k, random_state=i).fit(X)
        centers[k].append(kmeans.cluster_centers_)


In [ ]:
fig, axs = plt.subplots(2, 3)



for ax, (k, c),  in zip(axs.flat, centers.items()):
    ax.scatter(X[:, 0], X[:, 1], s=5)
    ax.set_aspect(1)
    c = np.array(c)
    ax.scatter(c[:, :, 0].ravel(), c[:, :, 1].ravel())
    ax.set_title(f'k = {k}')